In [1]:
# import tensorflow as tf
# tf.config.list_physical_devices('GPU')
# tf.test.is_built_with_cuda()
import os, sys
sys.path.append('../')
import torch
import torch.nn as nn 
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
from save_best_model import SaveBestModel
from sandpile import Sandpile, run_sandpile_alone
import random
from collections import deque
from torch.distributions import Categorical
import time
import datetime
from rl_agents import Policy
from agents import RLPolicyAgent
from util import Directions

# Set the seed value all over the place to make this reproducible.
seed_val = 42


random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


#RUN THIS ON COLAB
ON_COLAB = False
if ON_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/Phase ML Data/'

In [2]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()
model_nickname = 'reinforce-agent'

output_dir = f'/staging_area/{model_nickname}/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

checkpoint_dir = 'checkpoints/'
if not os.path.exists(output_dir+checkpoint_dir):
    os.makedirs(output_dir+checkpoint_dir)


best_model_name = 'best_agent.tar'
save_best_model = SaveBestModel(output_dir+best_model_name)

PermissionError: [Errno 13] Permission denied: '/staging_area'

In [3]:


def enum_parameters(model):
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        total_params+=params
    print(f"Total Trainable Params: {total_params}")
    return total_params



if torch.cuda.is_available():       
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")


if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(rl_policy)


# SET UP POLICY AGENT
N_grid = 10
num_hidden_layers = 2
hidden_dim = 64
input_dim = N_grid**2# The number of input variables. 
output_dim = len(Directions) # The number of output variables. 

rl_policy = Policy(
    input_dim=input_dim,
    num_hidden_layers=num_hidden_layers,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    device=device
)
enum_parameters(rl_policy)
rl_policy.to(device)

TypeError: __init__() missing 1 required positional argument: 'device'

In [4]:
MAXIMUM_GRAINS = 4
max_nmoves_per_episode = 10000

rl_policy_agent = RLPolicyAgent(rl_policy=rl_policy)
agents = [rl_policy_agent]

# start new sandpile with initial grid
sandpile = Sandpile(N_grid=N_grid, initial_grid=None, MAXIMUM_GRAINS=MAXIMUM_GRAINS, agents=agents, MAX_STEPS=10)
rl_policy.select_action(sandpile=sandpile)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [22]:

N_training_episodes = 100
N_val_episodes = 100
gamma = 0.99

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.998), eps=1e-9, weight_decay=1e-4)

start_epoch = 0
FROM_CHECKPOINT = not True
if FROM_CHECKPOINT:
    
    checkpoint = torch.load(output_dir+'best_gait_model.tar')
    g = checkpoint['model_state_dict']
    loss = checkpoint['loss']
    print(f'Lowest Loss: {loss}')
    save_best_model = SaveBestModel(output_dir+best_model_name, loss)
    # print(g.keys())
    model.load_state_dict(g)

    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    print(f'From epoch: {start_epoch}')



epochs = 5
SAVE_EVERY_EPOCH_N = 1

training_scores = []
validation_scores = []



# Measure the total training time for the whole run.
total_t0 = time.time()

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Measure how long the training epoch takes.
t0 = time.time()
for i_episode in range(1, N_training_episodes+1):
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('Training...')
    rl_policy.train()
    optimizer.zero_grad()   

    # Progress update every 40 batches.
    if i_episode % 100 == 0 and not i_episode == 0:
        # Calculate elapsed time in minutes.
        elapsed = format_time(time.time() - t0)
        
        # Report progress.
        print('  Episode {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(i_episode, len(N_training_episodes), elapsed))

    cumulative_scores_episode = []
    

    # generate initial grid
    # run the sandpile 1000 times
    initial_grid_N = N_grid * N_grid * 4
    # print('Generating initial grid')
    initial_grid = run_sandpile_alone(N_grid=N_grid, initial_grid=None, MAXIMUM_GRAINS=MAXIMUM_GRAINS, DROP_SAND=True, MAX_STEPS=initial_grid_N)
    print('initial grid')
    print(initial_grid)



    # start new sandpile with initial grid
    sandpile = Sandpile(N_grid=N_grid, initial_grid=initial_grid, MAXIMUM_GRAINS=MAXIMUM_GRAINS, agents=agents, MAX_STEPS=max_nmoves_per_episode)

    # move agent to random position at beginning of episode
    rl_policy_agent.move_agent_to_point(random.randint(0,N_grid-1), random.randint(0,N_grid-1))
    
    episode_rewards = []
    log_probs = []
    i = 0
    game_is_running = sandpile.step()
    while game_is_running:
        # print(i)
        i+=1
        sandpile_grid, agent_rewards, game_is_running = sandpile.step()

        # get action and log prob
        action = rl_policy_agent.action_idx
        log_prob = rl_policy_agent.log_prob

        print('action: ', action)
        print('log_prob: ', log_prob)


        #only one agent is running so agent_rewards is a list with one element
        reward = agent_rewards[0]
        log_probs.append(log_prob)

        episode_rewards.append(reward)

    cumulative_scores_episode.append(np.sum(episode_rewards))

    returns = deque(maxlen=max_nmoves_per_episode)
    n_steps_episode = len(episode_rewards)

    #TODO: replace with reverse numpy cumsum?
    for t in range(n_steps_episode)[::-1]:
        discounted_return_t = returns[0] if len(returns[0]) else 0
        returns.appendleft(gamma * discounted_return_t + episode_rewards[t])

    #standardize
    eps = np.finfo(np.float32).eps.item()
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    # compute loss
    policy_loss = []
    for log_prob, disc_return in zip(log_probs, returns):
        policy_loss.append(-log_prob * disc_return)
    policy_loss = torch.cat(policy_loss).sum()

    policy_loss.backward()
    optimizer.step()

    # Calculate the average loss over all of the batches.
    avg_train_score = np.mean(cumulative_scores_episode)     
    training_scores.append(avg_train_score)
    # Measure how long this episode took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training cumulative score: {0:.4f}".format(avg_train_score))
    print("  Training episode took: {:}".format(training_time))


    # ========================================
    #               Validation
    # ========================================
    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    rl_policy.eval()

    for episode_i in range(N_val_episodes):

        cumulative_scores_episode = []
    

        # generate initial grid
        # run the sandpile 1000 times
        initial_grid_N = N_grid * N_grid * 4
        # print('Generating initial grid')
        initial_grid = run_sandpile_alone(N_grid=N_grid, initial_grid=None, MAXIMUM_GRAINS=MAXIMUM_GRAINS, DROP_SAND=True, MAX_STEPS=initial_grid_N)
        print('initial grid')
        print(initial_grid)

        # start new sandpile with initial grid
    sandpile = Sandpile(N_grid=N_grid, initial_grid=initial_grid, MAXIMUM_GRAINS=MAXIMUM_GRAINS, agents=agents, MAX_STEPS=max_nmoves_per_episode)

    # move agent to random position at beginning of episode
    rl_policy_agent.move_agent_to_point(random.randint(0,N_grid-1), random.randint(0,N_grid-1))
    
    episode_rewards = []
    log_probs = []
    i = 0
    game_is_running = sandpile.step()
    while game_is_running:
        # print(i)
        i+=1
        sandpile_grid, agent_rewards, game_is_running = sandpile.step()

        # get action and log prob
        action = rl_policy_agent.action_idx
        log_prob = rl_policy_agent.log_prob

        print('action: ', action)
        print('log_prob: ', log_prob)


        #only one agent is running so agent_rewards is a list with one element
        reward = agent_rewards[0]
        log_probs.append(log_prob)

        episode_rewards.append(reward)

    cumulative_scores_episode.append(np.sum(episode_rewards))

    avg_val_score = np.mean(cumulative_scores_episode)


    validation_scores.append(avg_val_score)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Score: {0:.4f}".format(avg_val_score))
    print("  Validation took: {:}".format(validation_time))
    
    #save best model
    save_best_model(
        avg_val_score, episode_i+1, rl_policy, optimizer
    )
    
    #print training vals
    print('Validation scores')
    print(validation_scores)




print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

print("Saving model to %s" % output_dir)

#save model params
model_name = 'model_params.pt'
torch.save(model.state_dict(), output_dir+model_name)

model_name = 'model_full.pt'
torch.save(model, output_dir+model_name)


Training...
initial grid
[[2. 3. 2. 2. 3. 3. 2. 3. 3. 1.]
 [3. 3. 3. 2. 2. 1. 3. 2. 1. 2.]
 [2. 3. 1. 0. 2. 3. 2. 2. 0. 1.]
 [3. 3. 2. 2. 0. 2. 3. 3. 1. 3.]
 [3. 2. 3. 2. 3. 1. 3. 3. 3. 2.]
 [1. 3. 2. 1. 3. 2. 3. 2. 2. 1.]
 [2. 2. 0. 2. 2. 3. 3. 2. 1. 3.]
 [2. 1. 3. 3. 3. 3. 2. 0. 2. 3.]
 [2. 3. 2. 2. 0. 3. 2. 2. 2. 2.]
 [2. 1. 1. 1. 2. 3. 2. 0. 2. 3.]]


AttributeError: 'Policy' object has no attribute 'select_action'

In [ ]:
training_scores = np.array(training_scores)
validation_scores = np.array(validation_scores)
fig, axs = plt.subplots()
axs.plot(training_scores,'-',label='Train')
axs.set_ylabel('Scores')
axs.plot(validation_scores,'-',label='Val')
axs.set_xlabel('Episode')
axs.legend()
print(np.min(validation_scores))

In [ ]:
params = list(best_model.named_parameters())
print('The model has {:} different named parameters.\n'.format(len(params)))
for p in params:
    # print('p')
    # print(p[0])
    # print(p[1].data)
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(best_model)


In [ ]:
!gsutil cp -r ../staging_area/gait-model ../full_models/
!zip -r ../full_models/gait-model.zip ../full_models/gait-model/

In [ ]:
!gsutil cp -r ../full_models/gait-model/ gs://ml_gait_estimation/full_models/
!gsutil cp ../full_models/gait-model.zip gs://ml_gait_estimation/full_models/
